<a id="feature_engineering"></a>
# <p style="background-color: #87CEEB; font-family:calibri; color:black; font-size:140%; font-family:Verdana; text-align:center; border-radius:15px 50px;">Step 1 | Setup and Analysis</p>

# <b></span><span style='color:#87CEEB'> Importing Necessary Libraries</span></b>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# <b></span><span style='color:#87CEEB'> Load dataware house</span></b>

In [ ]:
df_flights = pd.read_csv('viagens/flights_ratings_modelos.csv')

In [ ]:
colunas_dataset = ["FL_DATE", "AIRLINE_CODE", "FL_NUMBER","ORIGIN","DEST","DEP_TIME","CRS_DEP_TIME","CRS_ARR_TIME",
                   "ARR_TIME","CANCELLED","DIVERTED","DISTANCE","DELAY_DUE_CARRIER","DELAY_DUE_WEATHER","DELAY_DUE_NAS",
                   "DELAY_DUE_SECURITY","DELAY_DUE_LATE_AIRCRAFT", "Rating", "Modelo Avião"]

In [ ]:
df_flights = df_flights[colunas_dataset]

# <b></span><span style='color:#87CEEB'> Initial Data Analysis</span></b>

In [ ]:
df_flights.head(10)

In [ ]:
df_flights.info()

In [ ]:
# Summary statistics for numerical variables
df_flights.describe().T

In [ ]:
# Summary statistics for categorical variables
df_flights.describe(include='object').T

<a id="feature_engineering"></a>
# <p style="background-color: #87CEEB; font-family:calibri; color:black; font-size:140%; font-family:Verdana; text-align:center; border-radius:15px 50px;">Step 2 | Data Cleaning</p>

# <b></span><span style='color:#87CEEB'> Treating Cancelled and Diverted Flights</span></b>

In [ ]:
df_cancelados = df_flights[df_flights["CANCELLED"] == 1.0].copy()
df_desviados = df_flights[df_flights["DIVERTED"] == 1.0].copy()

In [ ]:
diverted_percentage = (df_desviados.shape[0] / df_flights.shape[0]) * 100
cancelled_percentage = (df_cancelados.shape[0] / df_flights.shape[0]) * 100

In [ ]:
print(f"Cancelled flights%: {cancelled_percentage:.2f}\n Diverted flights%: {diverted_percentage:.2f}")

# <b></span><span style='color:#87CEEB'> Handling Nulls</span></b>

In [ ]:
# Calculating the percentage of missing values for each column
missing_data = df_flights.isnull().sum()
missing_percentage = (missing_data[missing_data > 0] / df_flights.shape[0]) * 100

# Prepare values
missing_percentage.sort_values(ascending=True, inplace=True)

# Plot the barh chart
fig, ax = plt.subplots(figsize=(15, 4))
ax.barh(missing_percentage.index, missing_percentage, color='#87CEEB')

# Annotate the values and indexes
for i, (value, name) in enumerate(zip(missing_percentage, missing_percentage.index)):
    ax.text(value+0.5, i, f"{value:.2f}%", ha='left', va='center', fontweight='bold', fontsize=18, color='black')

# Set x-axis limit
ax.set_xlim([0, 40])

# Add title and xlabel
plt.title("Percentage of Missing Values", fontweight='bold', fontsize=22)
plt.xlabel('Percentages (%)', fontsize=16)
plt.show()

In [ ]:
df_flights_clean = df_flights.dropna(subset=["ARR_TIME","DEP_TIME"]).copy()

In [ ]:
def hhmm_to_minutes(hhmm):
    # Converter valores nulos ou inválidos
    try:
        hhmm = int(hhmm)
        hours = hhmm // 100
        minutes = hhmm % 100
        return hours * 60 + minutes
    except:
        return None

# Converter os tempos para minutos
df_flights_clean.loc[:, "DEP_TIME_MIN"] =  df_flights_clean["DEP_TIME"].apply(hhmm_to_minutes)
df_flights_clean.loc[:, "CRS_DEP_TIME_MIN"] =  df_flights_clean["CRS_DEP_TIME"].apply(hhmm_to_minutes)
df_flights_clean.loc[:, "ARR_TIME_MIN"] =  df_flights_clean["ARR_TIME"].apply(hhmm_to_minutes)
df_flights_clean.loc[:, "CRS_ARR_TIME_MIN"] =  df_flights_clean["CRS_ARR_TIME"].apply(hhmm_to_minutes)

#for col in ["DEP_TIME", "CRS_DEP_TIME", "ARR_TIME", "CRS_ARR_TIME"]:
#    df_flights_clean[col + "_MIN"] = df_flights_clean[col].apply(hhmm_to_minutes)

# Calcular o TOTAL_DELAY
df_flights_clean["DEP_DELAY"] = (
    df_flights_clean["DEP_TIME_MIN"] - df_flights_clean["CRS_DEP_TIME_MIN"]
)

df_flights_clean["ARR_DELAY"] = (
    df_flights_clean["ARR_TIME_MIN"] - df_flights_clean["CRS_ARR_TIME_MIN"]
)


In [ ]:
df_flights_clean.head(10)

In [ ]:
df_cancelados1 = df_flights_clean[df_flights_clean["CANCELLED"] == 1.0].copy()
df_desviados1 = df_flights_clean[df_flights_clean["DIVERTED"] == 1.0].copy()

cancelled_percentage = (df_cancelados1.shape[0] / df_flights_clean.shape[0]) * 100
diverted_percentage = (df_desviados1.shape[0] / df_flights_clean.shape[0]) * 100

print(f"Cancelled flights%: {cancelled_percentage:.2f}\n Diverted flights%: {diverted_percentage:.2f}")


In [ ]:
df_flights_clean = df_flights_clean[(df_flights["DIVERTED"] != 1.0)].copy()

In [ ]:
df_flights_clean.drop(["CANCELLED", "DIVERTED", "DEP_TIME_MIN", "CRS_DEP_TIME_MIN", "ARR_TIME_MIN", "CRS_ARR_TIME_MIN"], axis=1, inplace=True)

In [ ]:
df_flights_clean.head(10)

<a id="feature_engineering"></a>
# <p style="background-color: #87CEEB; font-family:calibri; color:black; font-size:140%; font-family:Verdana; text-align:center; border-radius:15px 50px;">Step 3 | Feature Engineering</p>

# <b></span><span style='color:#87CEEB'> RFM Features</span></b>

<div style="border-radius:10px; padding:15px; background-color:#87CEEB; font-size:120%; text-align:left; color: black;">
  <h2>Recency</h2>
  <p>Vamos usar a <b>recência</b> para analisar quão recente foi o último atraso.</p>

  <h2>Frequency</h2>
  <p>A <b>frequência</b> nos permite descobrir se os atrasos são acontecimentos recorrentes.</p>

  <h2>Monetary</h2>
  <p>Qual é a <b>quantidade</b> total de atrasos? aplicar aos aeroportos? companhias aereas? aos dois? Nem sei se faz sentido aplicar isto aqui ou nao</p>
</div>

In [ ]:
# Convert FL_DATE to datetime type
df_flights_clean['FL_DATE'] = pd.to_datetime(df_flights_clean['FL_DATE'])

# Convert FL_DATE to datetime and extract only the date
df_flights_clean['FL_DATE'] = df_flights_clean['FL_DATE'].dt.date

# Find the most recent DELAY date for each airline
airline_data = df_flights_clean.groupby('AIRLINE_CODE')['FL_DATE'].max().reset_index()

# Find the most recent date in the entire dataset
most_recent_date = df_flights_clean['FL_DATE'].max()

# Convert FL_DATE to datetime type before subtraction
airline_data['FL_DATE'] = pd.to_datetime(airline_data['FL_DATE'])
most_recent_date = pd.to_datetime(most_recent_date)

# Calculate the number of days since the last purchase for each airline
airline_data['Days_Since_Last_Delay'] = (most_recent_date - airline_data['FL_DATE']).dt.days

# Remove the FL_DATE column
airline_data.drop(columns=['FL_DATE'], inplace=True)

In [ ]:
# Calculate the total number of flights made by each airline
total_flights = df_flights_clean.groupby('AIRLINE_CODE')['FL_DATE'].count().reset_index()
total_flights.rename(columns={'FL_DATE': 'Total_Flights'}, inplace=True)


# Merge the new features into the airline_data dataframe
airline_data = pd.merge(airline_data, total_flights, on='AIRLINE_CODE')

# Display the first few rows of the airline_data dataframe
airline_data.head()

In [ ]:
# Calculate the total delay by each airline
df_flights_clean['Total_Delay'] = df_flights_clean['DEP_DELAY'] + df_flights_clean['ARR_DELAY']
total_delay = df_flights_clean.groupby('AIRLINE_CODE')['Total_Delay'].sum().reset_index()

# Calculate the average transaction value for each customer
average_delay = total_delay.merge(total_flights, on='AIRLINE_CODE')
average_delay['Average_Delay'] = average_delay['Total_Delay'] / average_delay['Total_Flights']

# Merge the new features into the airline_data dataframe
airline_data = pd.merge(airline_data, total_delay, on='AIRLINE_CODE')
airline_data = pd.merge(airline_data, average_delay[['AIRLINE_CODE', 'Average_Delay']], on='AIRLINE_CODE')

# Display the first few rows of the airline_data dataframe
airline_data.head()

# <b></span><span style='color:#87CEEB'> Flight Diversity</span></b>

In [ ]:
# Calculate the number of unique products purchased by each airline
unique_flights = df_flights_clean.groupby('AIRLINE_CODE')['FL_NUMBER'].nunique().reset_index()
unique_flights.rename(columns={'FL_NUMBER': 'Unique_Flights'}, inplace=True)

# Merge the new feature into the airline_data dataframe
airline_data = pd.merge(airline_data, unique_flights, on='AIRLINE_CODE')

# Display the first few rows of the airline_data dataframe
airline_data.head()